In [1]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from RBM import RBM
from SVM import SVM
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 90
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

2022-05-11 19:48:54.968343 model.py code start


In [2]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
dataset_db1, dataset_db2 = mf.ecg_filtering(True)

train_dataset = list(mf.list_to_list(dataset_db1)) * 4
cross_dataset = list(mf.list_to_list(dataset_db2)) * 4
test_dataset = list(mf.list_to_list(dataset_db2))  * 4

X_train, X_test, y_train, y_test = train_test_split(
    (train_dataset + cross_dataset), 
    (test_dataset + cross_dataset),
    test_size=0.33,
    shuffle=True
)

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read file and indexing start...
[INFO]	final_db1 direcotry found.
......	...................i	Current_Index	From_Index
[IWIP]	final_db1 reading... 0 0 200
[IWIP]	final_db1 reading... 1 200 400
[IWIP]	final_db1 reading... 2 400 600
[IWIP]	final_db1 reading... 3 600 800
[IWIP]	final_db1 reading... 4 800 1000
[IWIP]	final_db1 reading... 5 1000 1200
[IWIP]	final_db1 reading... 6 1200 1400
[IWIP]	final_db1 reading... 7 1400 1600
[IWIP]	final_db1 reading... 8 1600 1800
[IWIP]	final_db1 reading... 9 1800 2000
[IWIP]	final_db1 reading... 10 2000 2200
[IWIP]	final_db1 reading... 11 2200 2400
[IWIP]	final_db1 reading... 12 2400 2600
[IWIP]	final_db1 reading... 13 2600 2800
[IWIP]	final_db1 reading... 14 2800 3000
[IWIP]	final_db1 reading... 15 3000 3200
[IWIP]	final_db1 reading... 16 3200 3400
[IWIP]	final_db1 reading... 17 3400 3600
[IWIP]	final_db1 reading... 18 

c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\scipy\signal\signaltools.py:1531: UserWarning: kernel_size exceeds volume extent: the volume will be zero-padded.
  warnings.warn('kernel_size exceeds volume extent: the volume will be '


[INFO]	final_db2 direcotry found.
[INFO]	final_db3 direcotry found.
[DONE] Pre-processing is done.


In [3]:
train_dataloader = DataLoader(X_train + y_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(X_test + y_test,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

print("X_train length : ", len(X_train))
print("X_test  length : ", len(X_test))
print("y_train length : ", len(y_train))
print("y_test  length : ", len(y_test))

train_data = torch.FloatTensor(X_train)
test_data = torch.FloatTensor(X_test)

X_train length :  6432
X_test  length :  3168
y_train length :  6432
y_test  length :  3168


In [4]:
print("[INFO] Model object added")

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

# first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
# second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
# third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

# gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
# gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
# gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

[INFO] Model object added


In [5]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for _, (data) in enumerate(train_dataloader):
        try:
            # tnesor float
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_second, v2, mt = rbm_second(sample_data)
        
        loss_second = rbm_second.free_energy(vog_second) - rbm_second.free_energy(v2)
        loss_.append(loss_second.data)
        
        second_train_op.zero_grad()
        loss_second.backward()
        second_train_op.step()

    output_from_second.append(v2.tolist())
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third bbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_third, v3, mt = rbm_third(sample_data)
        
        loss_third = rbm_third.free_energy(vog_third) - rbm_third.free_energy(v3)
        loss_.append(loss_third.data)
        
        third_train_op.zero_grad()
        loss_third.backward()
        third_train_op.step()

    output_from_third.append(v3.tolist())
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/3670610706.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1ST BBrbm_first Training loss for 0 epoch -0.10423406958580017	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 1 epoch -0.1129562109708786	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 2 epoch -0.14847776293754578	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 3 epoch -0.1572754681110382	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 4 epoch -0.11718638241291046	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 5 epoch -0.08992911875247955	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 6 epoch -0.10892117768526077	Estimate time : 0:00:00.001051
1ST BBrbm_first Training loss for 7 epoch -0.09430204331874847	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 8 epoch -0.0955905020236969	Estimate time : 0:00:00
1ST BBrbm_first Training loss for 9 epoch -0.08443785458803177	Estimate time : 0:00:00.000976
1ST BBrbm_first Training loss for 10 epoch -0.09081360697746277	Estimate time : 0:00:00
1ST BBrbm_first Training loss f

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/3670610706.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST BBrbm_first Training loss for 0 epoch -0.05965328961610794	Estimate time : 
2ST BBrbm_first Training loss for 1 epoch -0.05970938503742218	Estimate time : 
2ST BBrbm_first Training loss for 2 epoch -0.05966382473707199	Estimate time : 
2ST BBrbm_first Training loss for 3 epoch -0.06029503792524338	Estimate time : 
2ST BBrbm_first Training loss for 4 epoch -0.06029493734240532	Estimate time : 
2ST BBrbm_first Training loss for 5 epoch -0.06062810868024826	Estimate time : 
2ST BBrbm_first Training loss for 6 epoch -0.06083003804087639	Estimate time : 
2ST BBrbm_first Training loss for 7 epoch -0.0606277771294117	Estimate time : 
2ST BBrbm_first Training loss for 8 epoch -0.060756292194128036	Estimate time : 
2ST BBrbm_first Training loss for 9 epoch -0.06146038696169853	Estimate time : 
2ST BBrbm_first Training loss for 10 epoch -0.061489831656217575	Estimate time : 
2ST BBrbm_first Training loss for 11 epoch -0.062301356345415115	Estimate time : 
2ST BBrbm_first Training loss for 12

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/3670610706.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST BBrbm_first Training loss for 0 epoch -0.07462213188409805	Estimate time : 
3ST BBrbm_first Training loss for 1 epoch -0.07476483285427094	Estimate time : 
3ST BBrbm_first Training loss for 2 epoch -0.07530436664819717	Estimate time : 
3ST BBrbm_first Training loss for 3 epoch -0.0751611664891243	Estimate time : 
3ST BBrbm_first Training loss for 4 epoch -0.07506802678108215	Estimate time : 
3ST BBrbm_first Training loss for 5 epoch -0.07524630427360535	Estimate time : 
3ST BBrbm_first Training loss for 6 epoch -0.07533258944749832	Estimate time : 
3ST BBrbm_first Training loss for 7 epoch -0.07511084526777267	Estimate time : 
3ST BBrbm_first Training loss for 8 epoch -0.07521293312311172	Estimate time : 
3ST BBrbm_first Training loss for 9 epoch -0.0752677321434021	Estimate time : 
3ST BBrbm_first Training loss for 10 epoch -0.07562724500894547	Estimate time : 
3ST BBrbm_first Training loss for 11 epoch -0.07572095841169357	Estimate time : 
3ST BBrbm_first Training loss for 12 epo

In [6]:
    
print("BBRBM is done.")
print("GBRBM is start")

output_from_first = list()
output_from_second = list()
output_from_third = torch.tensor(output_from_third)

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

# print(output_from_third.size(), output_from_third.dim(), "\n", output_from_third)
gaussian_std = torch.arange(1, 0, -0.1)

for epoch in range(EPOCH):
    '''First gbrbm'''
    for _, (data) in enumerate(output_from_third):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_first, gb_v1, mt = rbm_first(sample_data)
        
        gb_loss_first = rbm_first.free_energy(gb_vog_first) - rbm_first.free_energy(gb_v1)
        loss_.append(gb_loss_first.data)
        
        gb_first_train_op.zero_grad()
        gb_loss_first.backward()
        gb_first_train_op.step()

    output_from_first.append(gb_v1.tolist())
    print("1ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Second gbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_second, gb_v2, mt = rbm_second(sample_data)
        
        gb_loss_second = rbm_second.free_energy(gb_vog_second) - rbm_second.free_energy(gb_v2)
        loss_.append(gb_loss_second.data)
        
        gb_second_train_op.zero_grad()
        gb_loss_second.backward()
        gb_second_train_op.step()

    output_from_second.append(gb_v2.tolist())
    print("2ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third gbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_third, gb_v3, mt = rbm_third(sample_data)
        
        gb_loss_third = rbm_third.free_energy(gb_vog_third) - rbm_second.free_energy(gb_v3)
        loss_.append(gb_loss_third.data)
        
        gb_third_train_op.zero_grad()
        gb_loss_third.backward()
        gb_third_train_op.step()

    print("3ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))


BBRBM is done.
GBRBM is start


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/2809123372.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


1ST GBrbm_first Training loss for 0 epoch -0.07587048411369324	Estimate time : 0:00:00.000996
1ST GBrbm_first Training loss for 1 epoch -0.07580403238534927	Estimate time : 0:00:00
1ST GBrbm_first Training loss for 2 epoch -0.0757375955581665	Estimate time : 0:00:00
1ST GBrbm_first Training loss for 3 epoch -0.07567139714956284	Estimate time : 0:00:00
1ST GBrbm_first Training loss for 4 epoch -0.07560500502586365	Estimate time : 0:00:00.000997
1ST GBrbm_first Training loss for 5 epoch -0.07554247975349426	Estimate time : 0:00:00
1ST GBrbm_first Training loss for 6 epoch -0.07547547668218613	Estimate time : 0:00:00.001001
1ST GBrbm_first Training loss for 7 epoch -0.07541342824697495	Estimate time : 0:00:00
1ST GBrbm_first Training loss for 8 epoch -0.07534836232662201	Estimate time : 0:00:00
1ST GBrbm_first Training loss for 9 epoch -0.07528682053089142	Estimate time : 0:00:00
1ST GBrbm_first Training loss for 10 epoch -0.07522326707839966	Estimate time : 0:00:00.000998
1ST GBrbm_first

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/2809123372.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST GBrbm_first Training loss for 0 epoch -0.07019393891096115	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 1 epoch -0.07014285027980804	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 2 epoch -0.0700925812125206	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 3 epoch -0.07004337757825851	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 4 epoch -0.06999409943819046	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 5 epoch -0.06994382292032242	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 6 epoch -0.06989458203315735	Estimate time : 0:00:00.001000
2ST GBrbm_first Training loss for 7 epoch -0.06984402239322662	Estimate time : 0:00:00.001002
2ST GBrbm_first Training loss for 8 epoch -0.0697929635643959	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 9 epoch -0.06974325329065323	Estimate time : 0:00:00
2ST GBrbm_first Training loss for 10 epoch -0.06969403475522995	Estimate time : 0:00:00
2ST GBrbm_first Training loss 

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/2809123372.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST GBrbm_first Training loss for 0 epoch -0.06586959958076477	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 1 epoch -0.06578019261360168	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 2 epoch -0.06569096446037292	Estimate time : 0:00:00.001020
3ST GBrbm_first Training loss for 3 epoch -0.06560123711824417	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 4 epoch -0.06551044434309006	Estimate time : 0:00:00.001000
3ST GBrbm_first Training loss for 5 epoch -0.06541945040225983	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 6 epoch -0.0653304010629654	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 7 epoch -0.06523957848548889	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 8 epoch -0.06515282392501831	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 9 epoch -0.06506530940532684	Estimate time : 0:00:00
3ST GBrbm_first Training loss for 10 epoch -0.06497836858034134	Estimate time : 0:00:00.000997
3ST GBrbm_first Traini

In [7]:
nprst = gb_v3.detach().numpy()
print(nprst)

rbm_first.get_weight()

[1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]


Parameter containing:
tensor([[ 1.4071e-02, -7.9139e-03, -7.6073e-03,  4.3864e-03,  1.2376e-05,
         -4.9374e-03,  2.7339e-02, -7.3691e-03, -6.2183e-03,  1.1650e-02]],
       requires_grad=True)

In [8]:
test_loss = 0
train_loss = 0
train_cnt = 0
summary_c = 0

for _, test_data in enumerate(test_dataloader):
    try:
        test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)
    except RuntimeError:
        pass
    
    testing_data = torch.flatten(torch.bernoulli(test_data))
    training_data = torch.flatten(torch.bernoulli(train_data))
    
    vt, vt1, _ = rbm_first(testing_data)
    test_loss = rbm_first.free_energy(vt) - rbm_first.free_energy(vt1)    
    
    vs, vs1, _ = rbm_first(training_data)
    train_loss = rbm_first.free_energy(vs) - rbm_first.free_energy(vs1)

    
    test_loss += torch.mean(torch.abs(vt1[vt1 >= 0] - vt[vt1 >= 0]))
    # print(vt1[vt1 >= 0] - vt[vt1 >= 0])
    summary_c += 1

print('Test loss : ' + str(test_loss / summary_c))
print('Train - test : ' + str(train_loss - test_loss))


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/200555237.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_9488/200555237.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)


Test loss : tensor(0.0008, grad_fn=<DivBackward0>)
Train - test : tensor(-0.5118, grad_fn=<SubBackward0>)


In [9]:
'''Test code'''
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

output_from_first = list()
output_from_second = list()
output_from_third = list()

test_loss = 0
epoch_cnt = 0

In [10]:
'''First BBRBM Guide Line'''

for _, data in enumerate(test_dataloader):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v1, vt1, _ = rbm_first(data)
    test_loss += rbm_first.free_energy(v1) - rbm_first.free_energy(vt1)
    epoch_cnt += 1
    output_from_first.append(vt1.tolist())
print('\tBBRBM_First_layer test loss : ', str(test_loss / epoch_cnt))

'''Second BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_first)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v2, vt2, _ = rbm_second(data)
    test_loss += rbm_second.free_energy(v2) - rbm_second.free_energy(vt2)
    epoch_cnt += 1
    output_from_second.append(vt2.tolist())
print('\tBBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

'''Third BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_second)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v3, vt3, _ = rbm_third(data)
    test_loss += rbm_third.free_energy(v3) - rbm_third.free_energy(vt3)
    epoch_cnt += 1
    output_from_third.append(vt3.tolist())
print('\tBBRBM_Third_layer test loss : ', str(test_loss / epoch_cnt))

	BBRBM_First_layer test loss :  tensor(-0.0074, grad_fn=<DivBackward0>)
	BBRBM_Second_layer test loss :  tensor(-0.0335, grad_fn=<DivBackward0>)
	BBRBM_Third_layer test loss :  tensor(-0.0348, grad_fn=<DivBackward0>)


In [11]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

output_from_first = list()
output_from_second = list()

test_loss = 0
epoch_cnt = 0

In [12]:
'''First GBRBM Guide Line'''
epoch_cnt = 0
for _, data in enumerate(torch.tensor(output_from_third)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.normal(mean=data, std=gaussian_std))

    v1, vt1, _ = rbm_first(sample_data)
    test_loss += rbm_first.free_energy(v1) - rbm_first.free_energy(vt1)
    epoch_cnt += 1
    output_from_first.append(vt1.tolist())
print('\tGBRBM_First_layer test loss : ', str(test_loss / epoch_cnt))

'''Second BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_first)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.normal(mean=data, std=gaussian_std))
    
    v2, vt2, _ = rbm_second(sample_data)
    test_loss += rbm_second.free_energy(v2) - rbm_second.free_energy(vt2)
    epoch_cnt += 1
    output_from_second.append(vt2.tolist())
print('\tGBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

'''Third BBRBM Guide Line'''

output_from_third = []
for _, data in enumerate(torch.tensor(output_from_second)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.bernoulli(test_data))
    
    v3, vt3, _ = rbm_third(sample_data)
    test_loss += rbm_third.free_energy(v3) - rbm_third.free_energy(vt3)
    epoch_cnt += 1
    output_from_third.append(vt3.tolist())
print('\tGBRBM_Third_layer test loss : ', str(test_loss / epoch_cnt))
print(test_loss)
print(epoch_cnt)

	GBRBM_First_layer test loss :  tensor(-0.0013, grad_fn=<DivBackward0>)
	GBRBM_Second_layer test loss :  tensor(-0.0003, grad_fn=<DivBackward0>)
	GBRBM_Third_layer test loss :  tensor(-0.0074, grad_fn=<DivBackward0>)
tensor(-14.1685, grad_fn=<AddBackward0>)
1902


In [14]:
svm_model = SVM()
optimizer = optim.SGD(svm_model.parameters(), lr=LEARNING_RATE)
svm_model.train()

X = test_loss
N = EPOCH

for epoch in range(EPOCH):
    perm = torch.randperm(EPOCH)
    sum_loss = 0

    for i in range(0, N, BATCH_SIZE):
        x = X[perm[i : i + BATCH_SIZE]]
        y = X[perm[i : i + BATCH_SIZE]]

        x = Variable(x)
        y = Variable(x)

        optimizer.zero_grad()
        output = svm_model(x)

        loss = torch.mean(torch.clamp(1 - output * y, min=0))
        loss.backward()
        optimizer.step()

        sum_loss += loss[0].data.cpu().numpy()
    print("Epoch : {}\tLoss : {}".format(epoch, sum_loss[0]))

TypeError: randperm() received an invalid combination of arguments - got (), but expected one of:
 * (int n, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int n, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
